# 제품 이상여부 판별 프로젝트

## 1. 데이터 불러오기


### 필수 라이브러리


In [1]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

### 데이터 읽어오기


In [2]:
ROOT_DIR = "data"
RANDOM_STATE = 110

# Load data
train = pd.read_csv(os.path.join(ROOT_DIR, "train_adv.csv"))
test = pd.read_csv('./data/test_adv.csv')

### 필요하거나 묶일 수 있는 변수 가져오기

In [3]:
train

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,...,Head Clean Position Y Collect Result_Fill2,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,2.5,-90,...,50,91.8,270.0,50,114.612,19.9,7.0,127,1,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,2.5,-90,...,50,91.8,270.0,50,85.000,19.6,7.0,185,1,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,12.5,90,...,50,91.8,270.0,50,114.612,19.8,10.0,73,1,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,12.5,90,...,50,91.8,270.0,50,85.000,19.9,12.0,268,1,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,2.5,-90,...,50,91.8,270.0,50,85.000,19.7,8.0,121,1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,2.5,-90,...,50,91.8,270.0,50,85.000,19.2,1.0,318,1,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,12.5,90,...,50,91.8,270.0,50,114.612,20.5,14.0,197,1,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,2.5,-90,...,50,91.8,270.0,50,85.000,19.7,1.0,27,1,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,12.5,90,...,50,91.8,270.0,50,85.000,20.1,13.0,117,1,Normal


In [4]:
# 이 셀은 한번만 클릭
train['Equipment_Dam'] = train['Equipment_Dam'].str.slice(15, 16)
train['Equipment_Fill1'] = train['Equipment_Fill1'].str.slice(17, 18)
train['Equipment_Fill2'] = train['Equipment_Fill2'].str.slice(17, 18)

test['Equipment_Dam'] = test['Equipment_Dam'].str.slice(15, 16)
test['Equipment_Fill1'] = test['Equipment_Fill1'].str.slice(17, 18)
test['Equipment_Fill2'] = test['Equipment_Fill2'].str.slice(17, 18)

In [5]:
# 새로운 영역 지정
use_train = pd.DataFrame(train['target'], columns = ['target'])
use_test = pd.DataFrame(test['Set ID'], columns = ['Set ID'])

In [6]:
# Dam, Fill1, Fill2에서 지정된 값이 다를 경우 Abnormal 
def inconsistant(data, columnname, iwantthiscolumnsname, is_train = True):
    # 장비 번호가 다르면 불일치
    if is_train:
        cri = [
            train[columnname + '_Dam'] != train[columnname + '_Fill1'],
            train[columnname + '_Dam'] != train[columnname + '_Fill2'],
            train[columnname + '_Fill1'] != train[columnname + '_Fill1'],
            data[iwantthiscolumnsname] == 1
        ]
        
    else:
        cri = [
            test[columnname + '_Dam'] != test[columnname + '_Fill1'],
            test[columnname + '_Dam'] != test[columnname + '_Fill2'],
            test[columnname + '_Fill1'] != test[columnname + '_Fill1'],
            data[iwantthiscolumnsname] == 1
        ]
    con = [1, 1, 1, 1]

    data[iwantthiscolumnsname] = np.select(cri, con, default = 0)

In [7]:
# 불일치 변수
use_train['inconsistant'] = 0
use_test['inconsistant'] = 0

# 기준
columnname = ['Equipment', 'Receip No Collect Result', 'Production Qty Collect Result', 'PalletID Collect Result', ]

# 장착
for i in columnname:
    inconsistant(use_train, i, 'inconsistant', True)
    inconsistant(use_test, i, 'inconsistant', False)

In [8]:
# 시간이 0이하, 900이상인 값은 이상치로 분류
for j in ['Machine Tact time Collect Result_Dam', 'Machine Tact time Collect Result_Fill1', 'Machine Tact time Collect Result_Fill2']:
    cri = [
        train[j] <= 0,
        train[j] > 900
    ]
    cri2 = [
        test[j] <= 0,
        test[j] > 900
    ]
    con = [
        1, 1
    ]
    use_train['inconsistant'] = np.select(cri, con, default = use_train['inconsistant'])
    use_test['inconsistant'] = np.select(cri2, con, default = use_test['inconsistant'])

In [9]:
# 유클리드 거리 계산
def euclide_distance(data, first_dot, second_dot):
    
    # 값 반환
    iwantcalculateeuclidedistance = np.array([0]*len(data))
    
    # 제곱값 더해주기
    for i, j in zip(first_dot, second_dot):
        iwantcalculateeuclidedistance += (data[i] - data[j])**2
        
    # 반환
    return iwantcalculateeuclidedistance**0.5

### stage by head

In [10]:
# dam , fill1
first_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam'
]

second_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1'
]

# 유클리드 계산을 통한 위치 차이
use_train['distance_diff_dam_fill1_stage1'] = euclide_distance(train, first_dot, second_dot)
use_test['distance_diff_dam_fill1_stage1'] = euclide_distance(test, first_dot, second_dot)

In [11]:
# fill1, fill2
first_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam'
]

second_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2',
    'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2',
    'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2'
]

# 유클리드 계산을 통한 위치 차이
use_train['distance_diff_dam_fill2_stage1'] = euclide_distance(train, first_dot, second_dot)
use_test['distance_diff_dam_fill2_stage1'] = euclide_distance(test, first_dot, second_dot)

In [12]:
# dam fill2
first_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1'
]

second_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2',
    'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2',
    'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2'
]

# 유클리드 계산을 통한 위치 차이
use_train['distance_diff_fill1_fill2_stage1'] = euclide_distance(train, first_dot, second_dot)
use_test['distance_diff_fill1_fill2_stage1'] = euclide_distance(test, first_dot, second_dot)

In [13]:
# dam , fill1
first_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam',
    'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam',
    'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam'
]

second_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1'
]

# 유클리드 계산을 통한 위치 차이
use_train['distance_diff_dam_fill1_stage2'] = euclide_distance(train, first_dot, second_dot)
use_test['distance_diff_dam_fill1_stage2'] = euclide_distance(test, first_dot, second_dot)

In [14]:
# fill1, fill2
first_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam',
    'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam',
    'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam'
]

second_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2',
    'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2',
    'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2'
]

# 유클리드 계산을 통한 위치 차이
use_train['distance_diff_dam_fill2_stage2'] = euclide_distance(train, first_dot, second_dot)
use_test['distance_diff_dam_fill2_stage2'] = euclide_distance(test, first_dot, second_dot)

In [15]:
# dam fill2
first_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1'
]

second_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2',
    'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2',
    'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2'
]

# 유클리드 계산을 통한 위치 차이
use_train['distance_diff_fill1_fill2_stage2'] = euclide_distance(train, first_dot, second_dot)
use_test['distance_diff_fill1_fill2_stage2'] = euclide_distance(test, first_dot, second_dot)

In [16]:
# dam , fill1
first_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam',
    'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam',
    'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam'
]

second_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1'
]

# 유클리드 계산을 통한 위치 차이
use_train['distance_diff_dam_fill1_stage3'] = euclide_distance(train, first_dot, second_dot)
use_test['distance_diff_dam_fill1_stage3'] = euclide_distance(test, first_dot, second_dot)

In [17]:
# fill1, fill2
first_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam',
    'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam',
    'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam'
]

second_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2',
    'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2',
    'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2'
]

# 유클리드 계산을 통한 위치 차이
use_train['distance_diff_dam_fill2_stage3'] = euclide_distance(train, first_dot, second_dot)
use_test['distance_diff_dam_fill2_stage3'] = euclide_distance(test, first_dot, second_dot)

In [18]:
# dam fill2
first_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1'
]

second_dot = [
    'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2',
    'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2',
    'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2'
]

# 유클리드 계산을 통한 위치 차이
use_train['distance_diff_fill1_fill2_stage3'] = euclide_distance(train, first_dot, second_dot)
use_test['distance_diff_fill1_fill2_stage3'] = euclide_distance(test, first_dot, second_dot)

### cure

In [19]:
# standby, start
first_dot = [
    'CURE STANDBY POSITION X Collect Result_Dam',
    'CURE STANDBY POSITION Z Collect Result_Dam'
]

second_dot = [
    'CURE START POSITION X Collect Result_Dam',
    'CURE START POSITION Z Collect Result_Dam'
]

# 유클리드 계산을 통한 위치 차이
train['dist_standby_start_cure_dam'] = euclide_distance(train, first_dot, second_dot)
test['dist_standby_start_cure_dam'] = euclide_distance(test, first_dot, second_dot)

In [20]:
# start, end
first_dot = [
    'CURE START POSITION X Collect Result_Dam',
    'CURE START POSITION Z Collect Result_Dam'
]

second_dot = [
    'CURE END POSITION X Collect Result_Dam',
    'CURE END POSITION Z Collect Result_Dam'
]

# 유클리드 계산을 통한 위치 차이
train['dist_start_end_cure_dam'] = euclide_distance(train, first_dot, second_dot)
test['dist_start_end_cure_dam'] = euclide_distance(test, first_dot, second_dot)

In [21]:
# standby, end
first_dot = [
    'CURE STANDBY POSITION X Collect Result_Dam',
    'CURE STANDBY POSITION Z Collect Result_Dam'
]

second_dot = [
    'CURE END POSITION X Collect Result_Dam',
    'CURE END POSITION Z Collect Result_Dam'
]

# 유클리드 계산을 통한 위치 차이
train['dist_standby_end_cure_dam'] = euclide_distance(train, first_dot, second_dot)
test['dist_standby_end_cure_dam'] = euclide_distance(test, first_dot, second_dot)

In [22]:
# cure_time
use_train['cure_time_dam'] = train['dist_start_end_cure_dam'] / train['CURE SPEED Collect Result_Dam']
use_test['cure_time_dam'] = test['dist_start_end_cure_dam'] / test['CURE SPEED Collect Result_Dam']

In [23]:
# standby, start
first_dot = [
    'CURE STANDBY POSITION X Collect Result_Fill2',
    'CURE STANDBY POSITION Z Collect Result_Fill2'
]

second_dot = [
    'CURE START POSITION X Collect Result_Fill2',
    'CURE START POSITION Z Collect Result_Fill2'
]

# 유클리드 계산을 통한 위치 차이
train['dist_standby_start_cure_fill2'] = euclide_distance(train, first_dot, second_dot)
test['dist_standby_start_cure_fill2'] = euclide_distance(test, first_dot, second_dot)

In [24]:
# start, end
first_dot = [
    'CURE START POSITION X Collect Result_Fill2',
    'CURE START POSITION Z Collect Result_Fill2'
]

second_dot = [
    'CURE END POSITION X Collect Result_Fill2',
    'CURE END POSITION Z Collect Result_Fill2'
]

# 유클리드 계산을 통한 위치 차이
train['dist_start_end_cure_fill2'] = euclide_distance(train, first_dot, second_dot)
test['dist_start_end_cure_fill2'] = euclide_distance(test, first_dot, second_dot)

In [25]:
# standby, end
first_dot = [
    'CURE STANDBY POSITION X Collect Result_Fill2',
    'CURE STANDBY POSITION Z Collect Result_Fill2'
]

second_dot = [
    'CURE END POSITION X Collect Result_Fill2',
    'CURE END POSITION Z Collect Result_Fill2'
]

# 유클리드 계산을 통한 위치 차이
train['dist_standby_end_cure_fill2'] = euclide_distance(train, first_dot, second_dot)
test['dist_standby_end_cure_fill2'] = euclide_distance(test, first_dot, second_dot)

In [26]:
# cure_time
use_train['cure_time_fill2'] = train['dist_start_end_cure_fill2'] / train['CURE SPEED Collect Result_Fill2']
use_test['cure_time_fill2'] = test['dist_start_end_cure_fill2'] / test['CURE SPEED Collect Result_Fill2']

In [27]:
# press21
use_train['press21_autoclave'] = np.where(train['2nd Pressure Collect Result_AutoClave'] - train['1st Pressure Collect Result_AutoClave'] > 0.3, 1, 0)
use_test['press21_autoclave'] = np.where(test['2nd Pressure Collect Result_AutoClave'] - test['1st Pressure Collect Result_AutoClave'] > 0.3, 1, 0)

In [28]:
# thon time
train['thon_time'] = train['Machine Tact time Collect Result_Dam'] - train['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam'] - train['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam'] - train['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']
test['thon_time'] = test['Machine Tact time Collect Result_Dam'] - test['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam'] - test['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam'] - test['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']

In [29]:
# 75% 이상이면 1 아니면 0
use_train['thon_time'] = np.where(train['thon_time'] >= 38.8, 1, 0)
use_test['thon_time'] = np.where(test['thon_time'] >= 38.8, 1, 0)

In [30]:
# Discharged 거리
use_train['discharged_dist_stage1_dam'] = train['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * train['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']
use_train['discharged_dist_stage2_dam'] = train['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * train['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam']
use_train['discharged_dist_stage3_dam'] = train['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * train['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']

use_test['discharged_dist_stage1_dam'] = test['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * test['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']
use_test['discharged_dist_stage2_dam'] = test['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * test['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam']
use_test['discharged_dist_stage3_dam'] = test['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * test['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']

In [31]:
# 기존 정보
use_train[['equip', 'dam_time', 'fill1_time', 'fill2_time', 'autoclave_time', 'pallet', 'qty', 'receip']] = train[['Equipment_Dam', 
                                                            'Machine Tact time Collect Result_Dam',
                                                            'Machine Tact time Collect Result_Fill1',
                                                            'Machine Tact time Collect Result_Fill2',
                                                            'Chamber Temp. Unit Time_AutoClave',
                                                            'PalletID Collect Result_Dam',
                                                            'Production Qty Collect Result_Dam',
                                                            'Receip No Collect Result_Dam']]
use_test[['equip', 'dam_time', 'fill1_time', 'fill2_time', 'autoclave_time', 'pallet', 'qty', 'receip']] = test[['Equipment_Dam',
                                                          'Machine Tact time Collect Result_Dam',
                                                          'Machine Tact time Collect Result_Fill1',
                                                          'Machine Tact time Collect Result_Fill2',
                                                          'Chamber Temp. Unit Time_AutoClave',
                                                          'PalletID Collect Result_Dam',
                                                          'Production Qty Collect Result_Dam',
                                                          'Receip No Collect Result_Dam']]

In [32]:
# 챔버 온도 영역 세개로 나누기
cri = [
    (train['Chamber Temp. Collect Result_AutoClave'] < 50) & (train['Chamber Temp. Judge Value_AutoClave'] == 'OK'),
    (train['Chamber Temp. Collect Result_AutoClave'] < 50) & (train['Chamber Temp. Judge Value_AutoClave'] == 'NG')
]
cri2 = [
    (test['Chamber Temp. Collect Result_AutoClave'] < 50) & (test['Chamber Temp. Judge Value_AutoClave'] == 'OK'),
    (test['Chamber Temp. Collect Result_AutoClave'] < 50) & (test['Chamber Temp. Judge Value_AutoClave'] == 'NG')
]
con = [
    1, 2
]

use_train['chamber_temp'] = np.select(cri, con, default = 0)
use_test['chamber_temp'] = np.select(cri2, con, default = 0)

In [33]:
# modelsuffix
le = LabelEncoder()

In [34]:
# 변환
use_train['model_suffix'] = le.fit_transform(train['Model.Suffix_Dam'])
use_test['model_suffix'] = le.transform(test['Model.Suffix_Dam'])

In [35]:
# 1, 0 으로 바꿔주기
use_train['target'] = np.where(train['target'] == 'Normal', 0, 1)

In [36]:
# set id 추가
use_test['Set ID'] = test['Set ID']

In [37]:
# 저장
use_train.to_csv('./data/train_0814_variable.csv', index = False)
use_test.to_csv('./data/test_0814_variable.csv', index = False)